In [807]:
import json
import numpy
import requests
from bs4 import BeautifulSoup
from espncricinfo.exceptions import MatchNotFoundError, NoScorecardError
import pandas as pd

class Match(object):
    def __init__(self, match_id):
        self.match_id = match_id
        self.match_url = f"https://site.api.espn.com/apis/site/v2/sports/cricket/8048/summary?event={match_id}"
        self.json_url = f"https://www.espncricinfo.com/matches/engine/match/{match_id}.json"
        self.json = self.get_json()
        self.json_api = self.get_json_api()
        self.description = self.get_description()
        self.full_scorecard_url = self.get_full_scorecard_url()
        self.toss = self.get_toss()
        self.home_team = self.get_home_team()
        self.away_team = self.get_away_team()
        self.batting_first = self.get_batting_first()
        self.batting_second = self.get_batting_second()
        # self.team = self.get_team()
        self.team1_batting_df = self.get_team1_batting_df()
        self.team2_batting_df = self.get_team2_batting_df()
        self.team1_bowling_df = self.get_team1_bowling_df()
        self.team2_bowling_df = self.get_team2_bowling_df()
        self.team1_dismissals_df = self.decider1()
        self.team2_dismissals_df = self.decider2()
        self.dream11_points = self.get_dream11_points()

        
    def get_json(self):
        r = requests.get(self.json_url)
        if r.status_code == 404:
            raise MatchNotFoundError
        elif 'Scorecard not yet available' in r.text:
            raise NoScorecardError
        else:
            return r.json()
    
    def get_json_api(self):
        r = requests.get(self.match_url)
        if r.status_code == 404:
            raise MatchNotFoundError
        elif 'Scorecard not yet available' in r.text:
            raise NoScorecardError
        else:
            return r.json()
        
    def get_full_scorecard_url(self):
        url  = self.description
        matchName = url.split(",")[1].split(":")[1].strip().split(" ")[:-2]
        firstHalf = '-'.join(matchName).lower().replace("-v-", "-vs-")
        matchNo = url.split(",")[1].split(":")[0].replace(" ","-").lower()
        finalurl = firstHalf + matchNo + "-" + str(self.match_id)
        return f"https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/{finalurl}/full-scorecard"
    
    def get_description(self):
        return self.json['description']
    
    def get_toss(self):
        return self.json_api['gameInfo']['venue']['fullName']

    
    def get_home_team(self):
        a = self.json_api['rosters'][0]['team']
        return {'name':a['abbreviation'], 'fullName' : a['displayName'], 'id' : a['id']}
    
    def get_away_team(self):
        a = self.json_api['rosters'][1]['team']
        return {'name':a['abbreviation'], 'fullName' : a['displayName'], 'id' : a['id']}
    
    # def get_team(self, team):
    #     team = []
    #     for i in self.json_api['rosters'][0]['roster']:
    #          team.append(i['athlete']['name'])
    #     return team
    
    def get_batting_first(self):
        a = self.json_api['matchcards'][0]
        teamName = a['teamName']
        otherTeamName = [self.home_team['name'],self.away_team['name']]
        otherTeamName.remove(teamName)
        inningsNumber = a['inningsNumber']
        headline = a['headline']
        if(headline == 'Batting'):
            if(inningsNumber == '1'):
                return teamName
            else:
                return otherTeamName[0]
        else:
            if(inningsNumber == '2'):
                return teamName
            else:
                return otherTeamName[0]

    def get_batting_second(self):
        a = self.json_api['matchcards'][0]
        teamName = a['teamName']
        otherTeamName = [self.home_team['name'],self.away_team['name']]
        otherTeamName.remove(teamName)
        inningsNumber = a['inningsNumber']
        headline = a['headline']
        if(headline == 'Batting'):
            if(inningsNumber == '1'):
                return otherTeamName[0]
            else:
                return teamName
        else:
            if(inningsNumber == '2'):
                return otherTeamName[0]
            else:
                return teamName

    def get_team1_batting_df(self):
        r = requests.get(self.full_scorecard_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        df = pd.DataFrame(columns= ['Name', "Wicket", "Runs", "Balls", "Fours", "Sixes"])
        tables = soup.find_all('table')
        brows = tables[0].findAll('tr')[1:-4]
        for brow in brows:
            x  = brow.findAll('td')
            if len(x) == 1:
                continue
            name = x[0].text
            name2 = name.replace('(c)','')
            name3 = name2.replace('†', '')
            name3 = name3.strip()
            df.loc[len(df)] = [name3, x[1].text, x[2].text, x[3].text, x[5].text, x[6].text]
        df[["Runs", "Balls", "Fours", "Sixes"]] = df[["Runs", "Balls", "Fours", "Sixes"]].astype(int)
        df["Strike Rate"] = (df["Runs"] / df["Balls"] * 100).round(2)
        return df

    def get_team2_batting_df(self):
        r = requests.get(self.full_scorecard_url)
        soup = BeautifulSoup(r.content, 'html.parser')
        df = pd.DataFrame(columns= ['Name', "Wicket", "Runs", "Balls", "Fours", "Sixes"])
        tables = soup.find_all('table')
        brows = tables[2].findAll('tr')[1:-4]
        for brow in brows:
            x  = brow.findAll('td')
            if len(x) == 1:
                continue
            name = x[0].text
            name2 = name.replace('(c)','')
            name3 = name2.replace('†', '')
            name3 = name3.strip()
            df.loc[len(df)] = [name3, x[1].text, x[2].text, x[3].text, x[5].text, x[6].text]
        df[["Runs", "Balls", "Fours", "Sixes"]] = df[["Runs", "Balls", "Fours", "Sixes"]].astype(int)
        df["Strike Rate"] = (df["Runs"] / df["Balls"] * 100).round(2)
        return df
    
    def get_team2_bowling_df(self):
        table = pd.read_html(self.full_scorecard_url)
        df = table[1]
        df = df[~df['BOWLING'].str[0].str.isdigit()]
        df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
        df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
        df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]] = df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]].astype("int")
        return df
    
    def get_team1_bowling_df(self):
        table = pd.read_html(self.full_scorecard_url)
        df = table[3]
        df = df[~df['BOWLING'].str[0].str.isdigit()]
        df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
        df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
        df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]] = df[["Maidens", "Runs", "Wickets", "0s", "4s", "6s", "WD", "NB"]].astype("int")
        return df
    
    def get_dream11_points(self):
        return None
    
    def get_team1_dismissals_df(self):
        df = pd.DataFrame(columns= ['Name', "Position", "Dismissal Type", "Bowler", "Catcher/Runout"])
        if self.home_team['name'] == self.batting_first:
            z = 0
        else:
            z = 1

        # if self.home_team['name'] == self.batting_second:
        #     a = 1
        # else:
        #     a = 0

        for i in self.json_api['rosters'][0]['roster']:
            name = i['athlete']['name']
            position = i['athlete']['position']['name']
            dismissalType = i['linescores'][z]['statistics']['categories'][0]['stats'][4]['value']
            try:
                if(len(i['linescores'][z]['statistics']['batting']['outDetails']['fielders']) > 1):
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName'] + '/' + i['linescores'][0]['statistics']['batting']['outDetails']['fielders'][1]['athlete']['displayName']
                else:
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName']
            except:
                catcher = None
            try:
                bowler = i['linescores'][z]['statistics']['batting']['outDetails']['bowler']['displayName']
            except:
                bowler = None
            df.loc[len(df)] = [name, position, dismissalType,bowler, catcher]
        return df
    
    def get_team2_dismissals_df(self):
        df = pd.DataFrame(columns= ['Name', "Position", "Dismissal Type", "Bowler", "Catcher/Runout"])
        if self.home_team['name'] == self.batting_first:
            z = 1
        else:
            z = 0

        # if self.home_team['name'] == self.batting_first:
        #     a = 1
        # else:
        #     a = 0 

        for i in self.json_api['rosters'][1]['roster']:
            name = i['athlete']['name']
            position = i['athlete']['position']['name']
            dismissalType = i['linescores'][z]['statistics']['categories'][0]['stats'][4]['value']
            try:
                if(len(i['linescores'][z]['statistics']['batting']['outDetails']['fielders']) > 1):
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName'] + '/' + i['linescores'][0]['statistics']['batting']['outDetails']['fielders'][1]['athlete']['displayName']
                else:
                    catcher = i['linescores'][z]['statistics']['batting']['outDetails']['fielders'][0]['athlete']['displayName']
            except:
                catcher = None
            try:
                bowler = i['linescores'][z]['statistics']['batting']['outDetails']['bowler']['displayName']
            except:
                bowler = None
            df.loc[len(df)] = [name, position, dismissalType,bowler, catcher]
        return df
    def decider1(self):
        if self.home_team['name'] == self.batting_first:
            return self.get_team1_dismissals_df()
        else:
            return self.get_team2_dismissals_df()
        
    def decider2(self):
        if self.home_team['name'] == self.batting_first:
            return self.get_team2_dismissals_df()
        else:
            return self.get_team1_dismissals_df()

In [808]:
matchNumber = 1426296
# t = Match(1426295)
m = Match(matchNumber)


/tmp/ipykernel_3642/3185414891.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
/tmp/ipykernel_3642/3185414891.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
/tmp/ipykernel_3642/3185414891.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [809]:
# x = m.team1_batting_df

# x['Fantasy Points'] = x['Runs'] + x['Fours'] + x['Sixes']*2
# x['Fantasy Points'] = x['Runs'] + x['Fours'] + x['Sixes']*2
# x['Fantasy Points'] = numpy.where(x['Runs'] > 50, x['Fantasy Points'] + 8, x['Fantasy Points'])
# x['Fantasy Points'] = numpy.where(x['Runs'] > 50, x['Fantasy Points'] + 8, x['Fantasy Points'])
# x['Fantasy Points'] = numpy.where(x['Runs'] > 100, x['Fantasy Points'] + 8, x['Fantasy Points'])
# x['Fantasy Points'] = numpy.where((x['Runs'] == 0) & (x['Wicket'] != 'not out '), x['Fantasy Points'] - 2 , x['Fantasy Points'])

# y = m.team1_bowling_df

# y['Fantasy Points'] = y['Wickets']*25 + y['Maidens']*8 
# y['Fantasy Points'] = numpy.where(y['Wickets'] >= 4, y['Fantasy Points'] + 8, y['Fantasy Points'])
# y['Fantasy Points'] = numpy.where(y['Wickets'] >= 5, y['Fantasy Points'] + 8, y['Fantasy Points'])
# y['Fantasy Points'] = numpy.where((y['Economy'] <= 6) & (y['Overs'] >=2) , y['Fantasy Points'] +2, y['Fantasy Points'])
# y['Fantasy Points'] = numpy.where((y['Economy'] <= 5) & (y['Overs'] >=2), y['Fantasy Points'] +2, y['Fantasy Points'])

# z = m.team2_dismissals_df

# df = pd.DataFrame(columns= ['Name', "Position", "Dismissal Type", "Bowler", "Catcher/Runout"])

# x

In [810]:
# z

In [811]:
# x = pd.merge(x,z)
# x['Fantasy Points'] = numpy.where((x['Strike Rate'] <=70 ) & (x['Balls'] >= 10) & (x['Position'] != 'Bowler'), x['Fantasy Points'] - 2 , x['Fantasy Points'])
# x['Fantasy Points'] = numpy.where((x['Strike Rate'] <60) & (x['Balls'] >= 10) & (x['Position'] != 'Bowler'), x['Fantasy Points'] - 2 , x['Fantasy Points'])
# x['Fantasy Points'] = numpy.where((x['Strike Rate'] <50 ) & (x['Balls'] >= 10) & (x['Position'] != 'Bowler'), x['Fantasy Points'] - 2 , x['Fantasy Points'])
# x.drop(x.columns[[8,9,10,11]], axis=1, inplace=True)
# x

In [812]:
# catcher = pd.DataFrame(columns= ['Name', "Position", "Dismissal Type", "Bowler", "Catcher/Runout"])
# catcher

In [813]:
match = Match(1426294)

/tmp/ipykernel_3642/3185414891.py:166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'O':'Overs', 'M' : 'Maidens', 'R' : 'Runs', 'W' : 'Wickets', 'ECON' : 'Economy'} , inplace= True)
/tmp/ipykernel_3642/3185414891.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[["Overs", "Economy",]] = df[["Overs","Economy"]].astype(float)
/tmp/ipykernel_3642/3185414891.py:168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [814]:
team1batting = match.team1_batting_df
team2batting = match.team2_batting_df
team1dismiss = match.team1_dismissals_df
team2dismiss = match.team2_dismissals_df
team1bowling = match.team1_bowling_df
team2bowling = match.team2_bowling_df

In [815]:
match.home_team

{'name': 'DC', 'fullName': 'Delhi Capitals', 'id': '335975'}

In [816]:
match.batting_first

'DC'

In [817]:
team1batting

,Name,Wicket,Runs,Balls,Fours,Sixes,Strike Rate
0,Jake Fraser-McGurk,c Ferreira b Ashwin,50,20,7,3,250.00
1,Abishek Porel,c Sandeep Sharma b Ashwin,65,36,7,3,180.56
2,Shai Hope,run out (Sandeep Sharma),1,1,0,0,100.00
3,Axar Patel,c Parag b Ashwin,15,10,1,1,150.00
4,Rishabh Pant,c Boult b Chahal,15,13,0,1,115.38
5,Tristan Stubbs,lbw b Sandeep Sharma,41,20,3,3,205.00
6,Gulbadin Naib,c Ashwin b Boult,19,15,1,1,126.67
7,Rasikh Salam,run out (Dubey/†Samson),9,3,2,0,300.00
8,Kuldeep Yadav,not out,5,2,1,0,250.00


In [818]:
team1dismiss
# problem = if home team batting first den this is wrong

,Name,Position,Dismissal Type,Bowler,Catcher/Runout
0,Jake Fraser-McGurk,Batter,c,Ravichandran Ashwin,Donovan Ferreira
1,Abishek Porel,Wicketkeeper batter,c,Ravichandran Ashwin,Sandeep Sharma
2,Shai Hope,Wicketkeeper batter,run out,None,Sandeep Sharma
3,Axar Patel,Bowling allrounder,c,Ravichandran Ashwin,Riyan Parag
4,Rishabh Pant,Wicketkeeper batter,c,Yuzvendra Chahal,Trent Boult
5,Tristan Stubbs,Batter,lbw,Sandeep Sharma,None
6,Gulbadin Naib,Batting allrounder,c,Trent Boult,Ravichandran Ashwin
7,Rasikh Salam,Bowler,run out,None,Shubham Dubey/Sanju Samson
8,Kuldeep Yadav,Bowler,not out,None,None
9,Mukesh Kumar,Bowler,0,None,None


In [819]:
team2batting

,Name,Wicket,Runs,Balls,Fours,Sixes,Strike Rate
0,Yashasvi Jaiswal,c Patel b Ahmed,4,2,1,0,200.00
1,Jos Buttler,b Patel,19,17,2,1,111.76
2,Sanju Samson,c Hope b Mukesh Kumar,86,46,8,6,186.96
3,Riyan Parag,b Rasikh Salam,27,22,1,3,122.73
4,Shubham Dubey,c Stubbs b Ahmed,25,12,2,2,208.33
5,Rovman Powell,b Mukesh Kumar,13,10,1,1,130.00
6,Donovan Ferreira,lbw b Kuldeep Yadav,1,3,0,0,33.33
7,Ravichandran Ashwin,c Hope b Kuldeep Yadav,2,3,0,0,66.67
8,Trent Boult,not out,2,3,0,0,66.67
9,Avesh Khan,not out,7,3,1,0,233.33


In [820]:
team2dismiss
# Problem vice versa

,Name,Position,Dismissal Type,Bowler,Catcher/Runout
0,Yashasvi Jaiswal,Opening batter,c,Khaleel Ahmed,Axar Patel
1,Jos Buttler,Wicketkeeper batter,0,Axar Patel,None
2,Sanju Samson,Wicketkeeper batter,c,Mukesh Kumar,Shai Hope
3,Riyan Parag,Top-order batter,0,Rasikh Salam,None
4,Shubham Dubey,Middle-order batter,c,Khaleel Ahmed,Tristan Stubbs
5,Rovman Powell,Middle-order batter,0,Mukesh Kumar,None
6,Donovan Ferreira,Allrounder,lbw,Kuldeep Yadav,None
7,Ravichandran Ashwin,Bowling allrounder,c,Kuldeep Yadav,Shai Hope
8,Trent Boult,Bowler,not out,None,None
9,Avesh Khan,Bowler,not out,None,None


In [821]:
# team1bowling

In [822]:
# team2bowling

In [823]:
def generate_batting_points(x, y):
    x = x.merge(y)
    x['Fantasy Points'] = x['Runs'] + x['Fours'] + x['Sixes']*2
    x['Fantasy Points'] = numpy.where(x['Runs'] >= 50, x['Fantasy Points'] + 8, x['Fantasy Points'])
    x['Fantasy Points'] = numpy.where(x['Runs'] >= 100, x['Fantasy Points'] + 8, x['Fantasy Points'])
    x['Fantasy Points'] = numpy.where((x['Runs'] == 0) & (x['Wicket'] != 'not out '), x['Fantasy Points'] - 2 , x['Fantasy Points'])
    x['Fantasy Points'] = numpy.where((x['Strike Rate'] <=70 ) & (x['Balls'] >= 10) & (x['Position'] != 'Bowler'), x['Fantasy Points'] - 2 , x['Fantasy Points'])
    x['Fantasy Points'] = numpy.where((x['Strike Rate'] <60) & (x['Balls'] >= 10) & (x['Position'] != 'Bowler'), x['Fantasy Points'] - 2 , x['Fantasy Points'])
    x['Fantasy Points'] = numpy.where((x['Strike Rate'] <50 ) & (x['Balls'] >= 10) & (x['Position'] != 'Bowler'), x['Fantasy Points'] - 2 , x['Fantasy Points'])
    x.drop(x.iloc[:, 1:11], inplace=True, axis=1)
    return x

def generate_bowling_points(y):
    y['Fantasy Points'] = y['Wickets']*25 + y['Maidens']*8 
    y['Fantasy Points'] = numpy.where(y['Wickets'] >= 4, y['Fantasy Points'] + 8, y['Fantasy Points'])
    y['Fantasy Points'] = numpy.where(y['Wickets'] >= 5, y['Fantasy Points'] + 8, y['Fantasy Points'])
    y['Fantasy Points'] = numpy.where((y['Economy'] <= 6) & (y['Overs'] >=2) , y['Fantasy Points'] +2, y['Fantasy Points'])
    y['Fantasy Points'] = numpy.where((y['Economy'] <= 5) & (y['Overs'] >=2), y['Fantasy Points'] +2, y['Fantasy Points'])
    y.drop(y.iloc[:, 1:11], inplace=True, axis=1)
    y.rename(columns = {'BOWLING':'Name'}, inplace = True)
    return y

# def generate_points(team1batting, team1dismiss, team2batting,team2dismiss, team1bowing, team2bowling):
    


team1batting = generate_batting_points(team1batting, team1dismiss)
team2batting = generate_batting_points(team2batting, team2dismiss)
team1bowling = generate_bowling_points(team1bowling)
team2bowling = generate_bowling_points(team2bowling)

# generate_points(team1batting, team1dismiss, team2batting,team2dismiss, team1bowling, team2bowling)

In [824]:
# pd.merge(team1batting , team1dismiss)
# team1bowling.groupby(['Name'],as_index=False).agg({'Fantasy Points': 'sum'})

In [825]:
# team1batting

In [826]:
# team1bowling

In [827]:
# team2batting

In [828]:
# team2bowling